In [1]:
# Romi Tshiorny
# Assignment 5
import pandas as pd
import seaborn as sb
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

#Load data
data = pd.read_csv("SouthGermanCredit.asc",delimiter = ' ')
data.head()

,laufkont,laufzeit,moral,verw,hoehe,sparkont,beszeit,rate,famges,buerge,...,verm,alter,weitkred,wohn,bishkred,beruf,pers,telef,gastarb,kredit
0,1,18,4,2,1049,1,2,4,2,1,...,2,21,3,1,1,3,2,1,2,1
1,1,9,4,0,2799,1,3,2,3,1,...,1,36,3,1,2,3,1,1,2,1
2,2,12,2,9,841,2,4,2,2,1,...,1,23,3,1,1,2,2,1,2,1
3,1,12,4,0,2122,1,3,3,3,1,...,1,39,3,1,2,2,1,1,1,1
4,1,12,4,0,2171,1,3,4,3,1,...,2,38,1,2,2,2,2,1,1,1


Not sure how to get 'table1' in from the example in here because it was done in R. I used 'pip install tableone' to use the python library instead.

In [2]:
#https://pypi.org/project/tableone/
from tableone import TableOne

mytable = TableOne(data)
print(mytable)

                      Missing          Overall
n                                         1000
laufkont, n (%)     1       0       274 (27.4)
                    2               269 (26.9)
                    3                 63 (6.3)
                    4               394 (39.4)
laufzeit, mean (SD)         0      20.9 (12.1)
moral, mean (SD)            0        2.5 (1.1)
verw, mean (SD)             0        2.8 (2.7)
hoehe, mean (SD)            0  3271.2 (2822.8)
sparkont, mean (SD)         0        2.1 (1.6)
beszeit, mean (SD)          0        3.4 (1.2)
rate, n (%)         1       0       136 (13.6)
                    2               231 (23.1)
                    3               157 (15.7)
                    4               476 (47.6)
famges, n (%)       1       0         50 (5.0)
                    2               310 (31.0)
                    3               548 (54.8)
                    4                 92 (9.2)
buerge, n (%)       1       0       907 (90.7)
             

In [13]:
#Split the data
X = data.drop(['kredit'],axis=1)
Y = data.drop(X.columns,axis=1)

X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.2,random_state=0)

C:\Users\Romi Tshiorny\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
#import and set up the models
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb

C:\Users\Romi Tshiorny\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [5]:
import warnings
from sklearn.exceptions import ConvergenceWarning
#For Scoring
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

#For cross-validation
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
#function to get various scores from a model
def get_scores(model,test_inputs,test_outputs):
    probs = model.predict(test_inputs)
    auc = roc_auc_score(probs,test_outputs)
    precision = precision_score(probs,test_outputs)
    recall = recall_score(probs,test_outputs)
    f1 = f1_score(probs,test_outputs)
    return round(auc,4), round(precision,4), round(recall,4), round(f1,4)
  
def Nfold_bestfit(model,train_inputs,train_outputs,k=10):
    kf = KFold(n_splits = k)
    best_train_X = None
    best_train_Y = None
    best_score = 0
    for train_index, test_index in kf.split(train_inputs):
        train_X = train_inputs.iloc[train_index]
        train_Y = train_outputs.iloc[train_index]
        test_X = train_inputs.iloc[test_index]
        test_Y = train_outputs.iloc[test_index]
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore", category=ConvergenceWarning,
                            module="sklearn")
            model.fit(train_X,train_Y.values.ravel())
        score = model.score(test_X,test_Y)
        if(score > best_score):
            best_score = score
            best_train_X = train_X
            best_train_Y = train_Y
            
    #Refit the model to the best fold
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", category=ConvergenceWarning,
                        module="sklearn")
        model.fit(best_train_X,best_train_Y.values.ravel())
        
def Nfold_bestfit_XGBoost(train_inputs,train_outputs,params,epochsk,k=10):
    kf = KFold(n_splits = k)
    best_train_X = None
    best_train_Y = None
    boost_model = None
    best_score = 0
    for train_index, test_index in kf.split(train_inputs):
        train_X = train_inputs.iloc[train_index]
        train_Y = train_outputs.iloc[train_index]
        test_X = train_inputs.iloc[test_index]
        test_Y = train_outputs.iloc[test_index]
        train_matrix = xgb.DMatrix(train_X,label=train_Y);
        test_matrix = xgb.DMatrix(test_X,label=test_Y);
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore", category=ConvergenceWarning,
                            module="sklearn")
            boost_model = xgb.train(params,train_matrix,epochs)
            
        probs = boost_model.predict(test_matrix)
        score = accuracy_score(probs,test_Y)
        if(score > best_score):
            best_score = score
            best_train_X = train_X
            best_train_Y = train_Y
            
    #Refit the model to the best fold
    train_matrix = xgb.DMatrix(best_train_X,label=best_train_Y);
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", category=ConvergenceWarning,
                        module="sklearn")
        boost_model = xgb.train(params,train_matrix,epochs)
    
    return boost_model
        
    

C:\Users\Romi Tshiorny\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
#Linear regression
regression_model = LogisticRegression(random_state=0)
#regression_model.fit(X_train,Y_train.values.ravel())
Nfold_bestfit(regression_model,X_train,Y_train)
regression_scores = get_scores(regression_model,X_test,Y_test)


C:\Users\Romi Tshiorny\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [7]:
#Decision Tree
tree_model = DecisionTreeClassifier(random_state=0)
Nfold_bestfit(tree_model,X_train,Y_train)
tree_scores = get_scores(tree_model,X_test,Y_test)

C:\Users\Romi Tshiorny\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [8]:
#Naive Bayes
naive_model = GaussianNB()
Nfold_bestfit(naive_model,X_train,Y_train)
naive_scores = get_scores(naive_model,X_test,Y_test)

C:\Users\Romi Tshiorny\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [9]:
#Random Forest
forest_model = RandomForestClassifier(random_state = 0)
Nfold_bestfit(forest_model,X_train,Y_train)
forest_scores = get_scores(forest_model,X_test,Y_test)

C:\Users\Romi Tshiorny\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [10]:
#XGBoost
train_matrix = xgb.DMatrix(X_train,label=Y_train);
test_matrix = xgb.DMatrix(X_test,label=Y_test);
params = {
    'max_depth':4,
    'eta':0.3,
    'objective':'multi:softmax',
    'disable_default_eval_metric':True,
    'num_class':2}
epochs = 10

boost_model = Nfold_bestfit_XGBoost(X_train,Y_train,params,epochs)
boost_scores = get_scores(boost_model,test_matrix,Y_test)

C:\Users\Romi Tshiorny\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [11]:
# Table for data
stats = [regression_scores,tree_scores,naive_scores,forest_scores,boost_scores]
columns = ['AUC','Precision','Recall','F-Score']
rows = ['LR','DT(%s)'%(tree_model.get_depth()),'NB','RF(default)','XGBoost(4,0.3)']
table = pd.DataFrame(np.array(stats),columns=columns,index=rows)
table.to_csv("scoretable.csv")
table


C:\Users\Romi Tshiorny\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,AUC,Precision,Recall,F-Score
LR,0.7450,0.9242,0.7531,0.8299
DT(20),0.6619,0.8030,0.7571,0.7794
NB,0.7496,0.8409,0.8222,0.8315
RF(default),0.7348,0.9167,0.7516,0.8259
"XGBoost(4,0.3)",0.8045,0.9621,0.7605,0.8495


In [12]:
from fairmlhealth import model_comparison as fhmc

model_dict = {'regression': regression_model,
             'tree': tree_model,
             'naive': naive_model,
             'forest': forest_model}


X = data.drop(['kredit'],axis=1)
Y = data.drop(X.columns,axis=1)

X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.2,random_state=0)

with warnings.catch_warnings():
        warnings.filterwarnings("ignore")
        
        #Conver to 0-1 binary for fhmc (male:0, female:1)
        X_test.loc[X_test['famges'] == 1,'famges'] = 0
        X_test.loc[X_test['famges'] == 2,'famges'] = 1
        X_test.loc[X_test['famges'] == 3,'famges'] = 0
        X_test.loc[X_test['famges'] == 4,'famges'] = 1

# Chose gender as a good protected attribute to evaluate fairness by
prtc_attr = X_test['famges']

X_test['pers']
fhmc.compare_measures(X_test,Y_test.values.ravel(),prtc_attr,model_dict)

C:\Users\Romi Tshiorny\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


regression  \
Group Fairness       Statistical Parity Difference               -0.0373   
                     Disparate Impact Ratio                       0.9543   
                     Demographic Parity Difference                0.0373   
                     Demographic Parity Ratio                     0.9543   
                     Average Odds Difference                     -0.0459   
                     Equal Opportunity Difference                -0.0517   
                     Equalized Odds Difference                    0.0517   
                     Equalized Odds Ratio                         0.9309   
                     Positive Predictive Parity Difference        0.0262   
                     Balanced Accuracy Difference                -0.0058   
                     AUC Difference                              -0.0917   
Individual Fairness  Consistency Score                            0.7700   
                     Between-Group Generalized Entropy Error      0.0004   
Model Performance ** Precision                                    0.7610   
                     Recall                                       0.9167   
                     F1-Score                                     0.8316   
                     Accuracy                                     0.7550   

                                                                tree   naive  \
Group Fairness       Statistical Parity Difference           -0.0269 -0.0526   
                     Disparate Impact Ratio                   0.9590  0.9085   
                     Demographic Parity Difference            0.0269  0.0526   
                     Demographic Parity Ratio                 0.9590  0.9085   
                     Average Odds Difference                 -0.0389 -0.0319   
                     Equal Opportunity Difference            -0.0263 -0.1457   
                     Equalized Odds Difference                0.0514  0.1457   
                     Equalized Odds Ratio                     0.8937  0.6631   
                     Positive Predictive Parity Difference    0.0378 -0.0525   
                     Balanced Accuracy Difference             0.0126 -0.1138   
                     AUC Difference                           0.0126 -0.1050   
Individual Fairness  Consistency Score                        0.6580  0.6410   
                     Between-Group Generalized Entropy Error  0.0004  0.0010   
Model Performance ** Precision                                0.7578  0.8716   
                     Recall                                   0.7348  0.7197   
                     F1-Score                                 0.7462  0.7884   
                     Accuracy                                 0.6700  0.7450   

                                                              forest  
Group Fairness       Statistical Parity Difference           -0.0347  
                     Disparate Impact Ratio                   0.9569  
                     Demographic Parity Difference            0.0347  
                     Demographic Parity Ratio                 0.9569  
                     Average Odds Difference                 -0.0291  
                     Equal Opportunity Difference            -0.0827  
                     Equalized Odds Difference                0.0827  
                     Equalized Odds Ratio                     0.9142  
                     Positive Predictive Parity Difference   -0.0013  
                     Balanced Accuracy Difference            -0.0536  
                     AUC Difference                          -0.0637  
Individual Fairness  Consistency Score                        0.7500  
                     Between-Group Generalized Entropy Error  0.0004  
Model Performance ** Precision                                0.7707  
                     Recall                                   0.9167  
                     F1-Score                                 0.8374  
                     Accuracy                

<h1>NOTE</h1>
I was not able get xgboost into the fairness evaluation due to how fhmc.compare_measures() takes its inputs. All the other models take in two seperate dataframe while xgboost takes one Dmatrix. I was not able to figure out a way to pass it into the method and have it work.